In [9]:
import os
import yfinance as yf
from datetime import datetime
import pandas as pd
import pytz
import logging

import matplotlib.pyplot as plt

In [10]:
import numpy as np
import plotly.graph_objects as go
from PIL import Image

In [28]:
def dta_to_candlestick(data):
    l = len(data)
    # Make candlestick picture
    layout = go.Layout(xaxis=dict(ticks='',
                                  showgrid=False,
                                  showticklabels=False,
                                  rangeslider=dict(visible=False)),
                       yaxis=dict(ticks='',
                                  showgrid=False,
                                  showticklabels=False),
                       width=256,
                       height=256,
                       paper_bgcolor='rgba(0,0,0,0)',
                       plot_bgcolor='rgba(0,0,0,0)')
    fig = go.Figure(data=[go.Candlestick(x=np.linspace(1,l,l),
                                         open=data.Open,
                                         high=data.High,
                                         low=data.Low,
                                         close=data.Close)],
                    layout=layout)
    fig.write_image("images/fig-33.png")

    # Convert to numpy array
    im = Image.open('images/fig-33.png')
    #im = im.resize((300,300),Image.ANTIALIAS)
    data = np.asarray(im)

    # Return the first channel of the image
    return data[:, :, 0]

In [29]:
START = datetime(1980, 1, 1)
END = datetime(2020, 4, 23)

In [30]:
tic = yf.Ticker('MSFT')
hist = tic.history(start=START, end=END)

In [31]:
def dta_transformation(data, est_h):
    # Make sure data has sufficient columns
    assert 'Open' in data.columns
    assert 'High' in data.columns
    assert 'Low' in data.columns
    assert 'Close' in data.columns

    data['lag_close'] = data['Close'].shift(1)
    data['Indicator'] = np.where(data['Close'] > data['lag_close'], 1, 0)

    x = []
    y = []
    for i in range(est_h, data.shape[0]):
        sub_dta = data.iloc[i - est_h:i]

        y_i = data.iloc[i]['Indicator']
        x_i = dta_to_candlestick(sub_dta)

        y.append(y_i)
        x.append(x_i)

        print("{}/{}".format(i - est_h, data.shape[0] - est_h))

    return x, y

In [39]:
x, y = dta_transformation(hist, 10)

0/8588
1/8588
2/8588
3/8588
4/8588
5/8588
6/8588
7/8588
8/8588
9/8588
10/8588
11/8588
12/8588
13/8588
14/8588
15/8588
16/8588
17/8588
18/8588
19/8588
20/8588
21/8588
22/8588
23/8588
24/8588
25/8588
26/8588
27/8588
28/8588
29/8588
30/8588
31/8588
32/8588
33/8588
34/8588
35/8588
36/8588
37/8588
38/8588
39/8588
40/8588
41/8588
42/8588
43/8588
44/8588
45/8588
46/8588
47/8588
48/8588
49/8588
50/8588
51/8588
52/8588
53/8588
54/8588
55/8588
56/8588
57/8588
58/8588
59/8588
60/8588
61/8588
62/8588
63/8588
64/8588
65/8588
66/8588
67/8588
68/8588
69/8588
70/8588
71/8588
72/8588
73/8588
74/8588
75/8588
76/8588
77/8588
78/8588
79/8588
80/8588
81/8588
82/8588
83/8588
84/8588
85/8588
86/8588
87/8588
88/8588
89/8588
90/8588
91/8588
92/8588
93/8588
94/8588
95/8588
96/8588
97/8588
98/8588
99/8588
100/8588
101/8588
102/8588
103/8588
104/8588
105/8588
106/8588
107/8588
108/8588
109/8588
110/8588
111/8588
112/8588
113/8588
114/8588
115/8588
116/8588
117/8588
118/8588
119/8588
120/8588
121/8588
122/8588
123

924/8588
925/8588
926/8588
927/8588
928/8588
929/8588
930/8588
931/8588
932/8588
933/8588
934/8588
935/8588
936/8588
937/8588
938/8588
939/8588
940/8588
941/8588
942/8588
943/8588
944/8588
945/8588
946/8588
947/8588
948/8588
949/8588
950/8588
951/8588
952/8588
953/8588
954/8588
955/8588
956/8588
957/8588
958/8588
959/8588
960/8588
961/8588
962/8588
963/8588
964/8588
965/8588
966/8588
967/8588
968/8588
969/8588
970/8588
971/8588
972/8588
973/8588
974/8588
975/8588
976/8588
977/8588
978/8588
979/8588
980/8588
981/8588
982/8588
983/8588
984/8588
985/8588
986/8588
987/8588
988/8588
989/8588
990/8588
991/8588
992/8588
993/8588
994/8588
995/8588
996/8588
997/8588
998/8588
999/8588
1000/8588
1001/8588
1002/8588
1003/8588
1004/8588
1005/8588
1006/8588
1007/8588
1008/8588
1009/8588
1010/8588
1011/8588
1012/8588
1013/8588
1014/8588
1015/8588
1016/8588
1017/8588
1018/8588
1019/8588
1020/8588
1021/8588
1022/8588
1023/8588
1024/8588
1025/8588
1026/8588
1027/8588
1028/8588
1029/8588
1030/8588
1031/8

1752/8588
1753/8588
1754/8588
1755/8588
1756/8588
1757/8588
1758/8588
1759/8588
1760/8588
1761/8588
1762/8588
1763/8588
1764/8588
1765/8588
1766/8588
1767/8588
1768/8588
1769/8588
1770/8588
1771/8588
1772/8588
1773/8588
1774/8588
1775/8588
1776/8588
1777/8588
1778/8588
1779/8588
1780/8588
1781/8588
1782/8588
1783/8588
1784/8588
1785/8588
1786/8588
1787/8588
1788/8588
1789/8588
1790/8588
1791/8588
1792/8588
1793/8588
1794/8588
1795/8588
1796/8588
1797/8588
1798/8588
1799/8588
1800/8588
1801/8588
1802/8588
1803/8588
1804/8588
1805/8588
1806/8588
1807/8588
1808/8588
1809/8588
1810/8588
1811/8588
1812/8588
1813/8588
1814/8588
1815/8588
1816/8588
1817/8588
1818/8588
1819/8588
1820/8588
1821/8588
1822/8588
1823/8588
1824/8588
1825/8588
1826/8588
1827/8588
1828/8588
1829/8588
1830/8588
1831/8588
1832/8588
1833/8588
1834/8588
1835/8588
1836/8588
1837/8588
1838/8588
1839/8588
1840/8588
1841/8588
1842/8588
1843/8588
1844/8588
1845/8588
1846/8588
1847/8588
1848/8588
1849/8588
1850/8588
1851/8588


2572/8588
2573/8588
2574/8588
2575/8588
2576/8588
2577/8588
2578/8588
2579/8588
2580/8588
2581/8588
2582/8588
2583/8588
2584/8588
2585/8588
2586/8588
2587/8588
2588/8588
2589/8588
2590/8588
2591/8588
2592/8588
2593/8588
2594/8588
2595/8588
2596/8588
2597/8588
2598/8588
2599/8588
2600/8588
2601/8588
2602/8588
2603/8588
2604/8588
2605/8588
2606/8588
2607/8588
2608/8588
2609/8588
2610/8588
2611/8588
2612/8588
2613/8588
2614/8588
2615/8588
2616/8588
2617/8588
2618/8588
2619/8588
2620/8588
2621/8588
2622/8588
2623/8588
2624/8588
2625/8588
2626/8588
2627/8588
2628/8588
2629/8588
2630/8588
2631/8588
2632/8588
2633/8588
2634/8588
2635/8588
2636/8588
2637/8588
2638/8588
2639/8588
2640/8588
2641/8588
2642/8588
2643/8588
2644/8588
2645/8588
2646/8588
2647/8588
2648/8588
2649/8588
2650/8588
2651/8588
2652/8588
2653/8588
2654/8588
2655/8588
2656/8588
2657/8588
2658/8588
2659/8588
2660/8588
2661/8588
2662/8588
2663/8588
2664/8588
2665/8588
2666/8588
2667/8588
2668/8588
2669/8588
2670/8588
2671/8588


3393/8588
3394/8588
3395/8588
3396/8588
3397/8588
3398/8588
3399/8588
3400/8588
3401/8588
3402/8588
3403/8588
3404/8588
3405/8588
3406/8588
3407/8588
3408/8588
3409/8588
3410/8588
3411/8588
3412/8588
3413/8588
3414/8588
3415/8588
3416/8588
3417/8588
3418/8588
3419/8588
3420/8588
3421/8588
3422/8588
3423/8588
3424/8588
3425/8588
3426/8588
3427/8588
3428/8588
3429/8588
3430/8588
3431/8588
3432/8588
3433/8588
3434/8588
3435/8588
3436/8588
3437/8588
3438/8588
3439/8588
3440/8588
3441/8588
3442/8588
3443/8588
3444/8588
3445/8588
3446/8588
3447/8588
3448/8588
3449/8588
3450/8588
3451/8588
3452/8588
3453/8588
3454/8588
3455/8588
3456/8588
3457/8588
3458/8588
3459/8588
3460/8588
3461/8588
3462/8588
3463/8588
3464/8588
3465/8588
3466/8588
3467/8588
3468/8588
3469/8588
3470/8588
3471/8588
3472/8588
3473/8588
3474/8588
3475/8588
3476/8588
3477/8588
3478/8588
3479/8588
3480/8588
3481/8588
3482/8588
3483/8588
3484/8588
3485/8588
3486/8588
3487/8588
3488/8588
3489/8588
3490/8588
3491/8588
3492/8588


...

5034/8588
5035/8588
5036/8588
5037/8588
5038/8588
5039/8588
5040/8588
5041/8588
5042/8588
5043/8588
5044/8588
5045/8588
5046/8588
5047/8588
5048/8588
5049/8588
5050/8588
5051/8588
5052/8588
5053/8588
5054/8588
5055/8588
5056/8588
5057/8588
5058/8588
5059/8588
5060/8588
5061/8588
5062/8588
5063/8588
5064/8588
5065/8588
5066/8588
5067/8588
5068/8588
5069/8588
5070/8588
5071/8588
5072/8588
5073/8588
5074/8588
5075/8588
5076/8588
5077/8588
5078/8588
5079/8588
5080/8588
5081/8588
5082/8588
5083/8588
5084/8588
5085/8588
5086/8588
5087/8588
5088/8588
5089/8588
5090/8588
5091/8588
5092/8588
5093/8588
5094/8588
5095/8588
5096/8588
5097/8588
5098/8588
5099/8588
5100/8588
5101/8588
5102/8588
5103/8588
5104/8588
5105/8588
5106/8588
5107/8588
5108/8588
5109/8588
5110/8588
5111/8588
5112/8588
5113/8588
5114/8588
5115/8588
5116/8588
5117/8588
5118/8588
5119/8588
5120/8588
5121/8588
5122/8588
5123/8588
5124/8588
5125/8588
5126/8588
5127/8588
5128/8588
5129/8588
5130/8588
5131/8588
5132/8588
5133/8588


5854/8588
5855/8588
5856/8588
5857/8588
5858/8588
5859/8588
5860/8588
5861/8588
5862/8588
5863/8588
5864/8588
5865/8588
5866/8588
5867/8588
5868/8588
5869/8588
5870/8588
5871/8588
5872/8588
5873/8588
5874/8588
5875/8588
5876/8588
5877/8588
5878/8588
5879/8588
5880/8588
5881/8588
5882/8588
5883/8588
5884/8588
5885/8588
5886/8588
5887/8588
5888/8588
5889/8588
5890/8588
5891/8588
5892/8588
5893/8588
5894/8588
5895/8588
5896/8588
5897/8588
5898/8588
5899/8588
5900/8588
5901/8588
5902/8588
5903/8588
5904/8588
5905/8588
5906/8588
5907/8588
5908/8588
5909/8588
5910/8588
5911/8588
5912/8588
5913/8588
5914/8588
5915/8588
5916/8588
5917/8588
5918/8588
5919/8588
5920/8588
5921/8588
5922/8588
5923/8588
5924/8588
5925/8588
5926/8588
5927/8588
5928/8588
5929/8588
5930/8588
5931/8588
5932/8588
5933/8588
5934/8588
5935/8588
5936/8588
5937/8588
5938/8588
5939/8588
5940/8588
5941/8588
5942/8588
5943/8588
5944/8588
5945/8588
5946/8588
5947/8588
5948/8588
5949/8588
5950/8588
5951/8588
5952/8588
5953/8588


6675/8588
6676/8588
6677/8588
6678/8588
6679/8588
6680/8588
6681/8588
6682/8588
6683/8588
6684/8588
6685/8588
6686/8588
6687/8588
6688/8588
6689/8588
6690/8588
6691/8588
6692/8588
6693/8588
6694/8588
6695/8588
6696/8588
6697/8588
6698/8588
6699/8588
6700/8588
6701/8588
6702/8588
6703/8588
6704/8588
6705/8588
6706/8588
6707/8588
6708/8588
6709/8588
6710/8588
6711/8588
6712/8588
6713/8588
6714/8588
6715/8588
6716/8588
6717/8588
6718/8588
6719/8588
6720/8588
6721/8588
6722/8588
6723/8588
6724/8588
6725/8588
6726/8588
6727/8588
6728/8588
6729/8588
6730/8588
6731/8588
6732/8588
6733/8588
6734/8588
6735/8588
6736/8588
6737/8588
6738/8588
6739/8588
6740/8588
6741/8588
6742/8588
6743/8588
6744/8588
6745/8588
6746/8588
6747/8588
6748/8588
6749/8588
6750/8588
6751/8588
6752/8588
6753/8588
6754/8588
6755/8588
6756/8588
6757/8588
6758/8588
6759/8588
6760/8588
6761/8588
6762/8588
6763/8588
6764/8588
6765/8588
6766/8588
6767/8588
6768/8588
6769/8588
6770/8588
6771/8588
6772/8588
6773/8588
6774/8588


7495/8588
7496/8588
7497/8588
7498/8588
7499/8588
7500/8588
7501/8588
7502/8588
7503/8588
7504/8588
7505/8588
7506/8588
7507/8588
7508/8588
7509/8588
7510/8588
7511/8588
7512/8588
7513/8588
7514/8588
7515/8588
7516/8588
7517/8588
7518/8588
7519/8588
7520/8588
7521/8588
7522/8588
7523/8588
7524/8588
7525/8588
7526/8588
7527/8588
7528/8588
7529/8588
7530/8588
7531/8588
7532/8588
7533/8588
7534/8588
7535/8588
7536/8588
7537/8588
7538/8588
7539/8588
7540/8588
7541/8588
7542/8588
7543/8588
7544/8588
7545/8588
7546/8588
7547/8588
7548/8588
7549/8588
7550/8588
7551/8588
7552/8588
7553/8588
7554/8588
7555/8588
7556/8588
7557/8588
7558/8588
7559/8588
7560/8588
7561/8588
7562/8588
7563/8588
7564/8588
7565/8588
7566/8588
7567/8588
7568/8588
7569/8588
7570/8588
7571/8588
7572/8588
7573/8588
7574/8588
7575/8588
7576/8588
7577/8588
7578/8588
7579/8588
7580/8588
7581/8588
7582/8588
7583/8588
7584/8588
7585/8588
7586/8588
7587/8588
7588/8588
7589/8588
7590/8588
7591/8588
7592/8588
7593/8588
7594/8588


8316/8588
8317/8588
8318/8588
8319/8588
8320/8588
8321/8588
8322/8588
8323/8588
8324/8588
8325/8588
8326/8588
8327/8588
8328/8588
8329/8588
8330/8588
8331/8588
8332/8588
8333/8588
8334/8588
8335/8588
8336/8588
8337/8588
8338/8588
8339/8588
8340/8588
8341/8588
8342/8588
8343/8588
8344/8588
8345/8588
8346/8588
8347/8588
8348/8588
8349/8588
8350/8588
8351/8588
8352/8588
8353/8588
8354/8588
8355/8588
8356/8588
8357/8588
8358/8588
8359/8588
8360/8588
8361/8588
8362/8588
8363/8588
8364/8588
8365/8588
8366/8588
8367/8588
8368/8588
8369/8588
8370/8588
8371/8588
8372/8588
8373/8588
8374/8588
8375/8588
8376/8588
8377/8588
8378/8588
8379/8588
8380/8588
8381/8588
8382/8588
8383/8588
8384/8588
8385/8588
8386/8588
8387/8588
8388/8588
8389/8588
8390/8588
8391/8588
8392/8588
8393/8588
8394/8588
8395/8588
8396/8588
8397/8588
8398/8588
8399/8588
8400/8588
8401/8588
8402/8588
8403/8588
8404/8588
8405/8588
8406/8588
8407/8588
8408/8588
8409/8588
8410/8588
8411/8588
8412/8588
8413/8588
8414/8588
8415/8588


In [40]:
x = np.stack(x, axis=2)

In [41]:
x.shape

(256, 256, 8588)

In [35]:
im = Image.fromarray(x[:,:,0])
im.show()

In [36]:
x[:,:,0]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [37]:
sub_x = x[:,:,568:1568]

In [38]:
y = np.array(y)
y.shape

(30,)

In [24]:
sub_y = y[568:1568]

In [25]:
np.savez_compressed('test_1', x=sub_x, y=sub_y)

In [27]:
x.shape[2] // 1000

8

In [42]:
for i in range(8):
    sub_x = x[:,:,(1000*i+568):(1000*(i+1)+568)]
    sub_y = y[(1000*i+568):(1000*(i+1)+568)]
    np.savez_compressed('test_{}'.format(i), x=sub_x, y=sub_y)

In [19]:
350000000/1024/1024

333.7860107421875

In [17]:
x.shape

2998800000

In [11]:
loaded = np.load('test_0.npz')

In [12]:
dta_x = loaded['x']

In [13]:
dta_x.shape

(256, 256, 1000)

In [14]:
dta_y = loaded['y']
dta_y.shape

(1000,)

## Pytorch Machine Learning Shit

In [1]:
from ResNet_CNN import *

In [2]:
import math as m
import torch
from torch.nn import Linear, ReLU, Conv1d, Conv2d, Flatten, Sequential, CrossEntropyLoss, MSELoss, MaxPool1d, MaxPool2d, Dropout, BatchNorm1d, BatchNorm2d

from torch.optim import Adam
from torch import nn
import torchvision

In [3]:
from functools import partial
from collections import OrderedDict

In [4]:
import torchvision.models as models

In [5]:
from torchsummary import summary

In [6]:
model = res_conv1(1, 64)

In [7]:
print(model)

ResNet(
  (encoder): ResNetEncoder(
    (gate): Sequential(
      (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01, inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (blocks): ModuleList(
      (0): ResNetLayer(
        (blocks): Sequential(
          (0): ResNetBottleNeckBlock(
            (blocks): Sequential(
              (0): Sequential(
                (0): Conv2dAuto(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (1): LeakyReLU(negative_slope=0.01, inplace=True)
              (2): Sequential(
                (0): Conv2dAuto(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (1): BatchNo

## Putting everything together

In [15]:
X = dta_x

In [16]:
Y = dta_y

In [32]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [17]:
import torchvision.transforms as transforms

In [18]:
trans = transforms.Compose([transforms.ToTensor(),
                           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [20]:
from torch.autograd import Variable
from torch.optim import Adam, SGD

In [21]:
X = X.reshape((-1, 256, 256)).astype(np.float32)
Y = Y.reshape((-1, 1)).astype(np.float32)

In [22]:
X.shape

(1000, 256, 256)

In [23]:
Y.shape

(1000, 1)

In [24]:
train_X, val_X, train_Y, val_Y = train_test_split(X, Y, test_size=0.1, random_state=42)

In [25]:
(train_X.shape, train_Y.shape), (val_X.shape, val_Y.shape)

(((900, 256, 256), (900, 1)), ((100, 256, 256), (100, 1)))

In [26]:
train_X = train_X.reshape(-1, 1, 256, 256)
train_X = torch.from_numpy(train_X)

train_Y = train_Y.astype(int)
train_Y = torch.from_numpy(train_Y)

In [27]:
val_X =  val_X.reshape(-1, 1, 256, 256)
val_X = torch.from_numpy(val_X)

val_Y = val_Y.astype(int)
val_Y = torch.from_numpy(val_Y)

In [28]:
model = res_conv1(1, 64)

In [29]:
lr = 0.01
optimizer = Adam(model.parameters(), lr=lr)

criterion = nn.BCELoss()

In [30]:
def train(epochs):
    model.train()
    tr_loss = 0
    # dataset
    x_train, y_train = Variable(train_X), Variable(train_Y)
    x_val, y_val = Variable(val_X), Variable(val_Y)
 
    optimizer.zero_grad()
    
    output_train = model(x_train)
    output_val = model(x_val)
    
    loss_train = criterion(output_train, y_train.type(torch.float))
    loss_val = criterion(output_val, y_val.type(torch.float))
    
    train_losses.append(loss_train)
    val_losses.append(loss_val)
    
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()
    
    print('Epoch: ', epochs+1, '\t', 'train loss: ', loss_train, '\t', 'val loss: ', loss_val)

In [31]:
x_train, y_train = Variable(train_X), Variable(train_Y)
x_val, y_val = Variable(val_X), Variable(val_Y)
optimizer.zero_grad()

In [32]:
output_train = model(x_train)

In [33]:
output_val = model(x_val)

In [34]:
loss_train = criterion(output_train, y_train.type(torch.float))
loss_val = criterion(output_val, y_val.type(torch.float))

In [35]:
loss_train.backward()

In [ ]:
n_epochs = 2
train_losses = []
val_losses = []

for epochs in range(n_epochs):
    train(epochs)